# Model Description 

I'm going to build a small model of the human body that uses a lummped capacitence to
deterimne the temperature of the human body.

The model I am going to use is going to be composed of nodes that are going to be
coupled together as a set of energy balances, with a number of parameters. The segemnts
(each of which will have an energy balance equation associated with it) are going to be
the head, torso, and left and right upper arms, lower arms, hands, thighs, lower legs, 
and feet. Each segment will be one node

For each of the segments, the energy balance equation is going toe be of the form:  
$q_{sh}+q_{met}+q_{bl,in}=q_{bl,out}+q_{env}+q_{store}$


In [2]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

## Model terms

1. $q_{sh}=Constant$, for most of our models (as a base) this will be 0
2. $q_{met}= Constant$, constant for our steady state model
3. $q_{in}=\dot{m}C_p(T-T_{s-1})$
4. $q_{out}=\dot{m}C_p(T_{s+1}-T)$
5. $q_{env}=(T-T_{inf})/R$
6. $q_{store}=\rho{}Vc(T-T_{old})/\Delta{}T$

*Note: I am going to have to make sure that all of these equations are either
in terms of energy (so multiplied by a time step in some way) or in terms of the 
heat flux at each time step*

## Terms to Matrix

Each of the above terms is able to be recast into a set 
of equations of the form:  
$\sum(C_{m,j}+C_{m,j}+C_{m,j})T_i=C$  
such that for each node in the system, the heat flow into and
out of the node (and stored in the node) is balanced. When the system
has reached equalibrium, this should be stable with time. The $C$ values (from all the
temperature independant terms)

This means that when building a matrix, any set of effects 
can be seen as adding parameters to the matrix such that the temperatures
from these paramters enter the matrix in a rational way. In the optimal case
The matrix will be chosen to be some form that can be solved easily. However since
the system is small, this is not important

## Node construction

Each node is going to have a temperature, an a set of (posibly time and temperature depened parameters, which will be
associated with temperature indexes, perhaps (as a matter of location in a tree))

## Container

Contains a tree that holds all the body parts and then can construct the thermal conductivity matrix

# First Pass 

For a first pass I am going to manual build the thermal conductivity matrix:


## The matrix 

The matrix equation is 

$A*t=0$ 
where A is the matrix for the energy balance equations, and t is a vector the temperatures of the form

1. Head
2. Body
3. Left upper arm
4. Left lower arm
5. Left hand
6. Right upper arm
7. Right lower arm
8. Right hand
9. Left upper leg
10. Left lower leg
11. Left foot
12. Right upper leg
13. Right lower leg
14. Right foot 

*This is the same setup as the tanabe model, and thus we should have a setup with a 
14 x 14 Matrix, with each row being for a single node*

In [24]:
bmm = np.zeros((14,14))

In [25]:
#All the coefficents
# first, the storage coefficents, which will end up being of the
# form ρVc

# I am going to make these strings to debug the matrix as a first pass

eta_0 = 1
eta_1, eta_2, eta_3, eta_4 = 1, 2, 3, 4
eta_5, eta_6, eta_7, eta_8 = 1, 2, 3, 4
eta_9, eta_10, eta_11, eta_12 = 1, 2, 3, 4
eta_13 = 1


In [26]:
# Enviroment Coefficents

k_0 = 1
k_1, k_2, k_3, k_4 = 1, 2, 3, 4
k_5, k_6, k_7, k_8 = 1, 2, 3, 4
k_9, k_10, k_11, k_12 = 1, 2, 3, 4
k_13 = 1


In [27]:
# metabolic rate Coefficents

q_0 = 1
q_1, q_2, q_3, q_4 = 1, 2, 3, 4
q_5, q_6, q_7, q_8 = 1, 2, 3, 4
q_9, q_10, q_11, q_12 = 1, 2, 3, 4
q_13 = 1


In [28]:
# blood coefficents

xi_01, xi_21, xi_51, xi_81, xi_11_1 = 1, 2, 3, 4, 5
xi_32, xi_43 = 1, 2
xi_65, xi_76 = 1, 2
xi_98, xi_10_9 = 1, 2
xi_12_11, xi_13_12 = 1, 2


In [29]:
dt = 0.1

In [30]:
### And build the matrix

bmm[0,0] = eta_0 / dt - xi_01 + k_0 
bmm[1,0], bmm[0,1] = xi_01, xi_01
bmm[1,1] = eta_1 / dt - xi_01 - xi_21 - xi_51 - xi_81 - xi_11_1 + k_1
bmm[2,1], bmm[1,2] = xi_21, xi_21
bmm[5,1], bmm[1,5] = xi_51, xi_51
bmm[8,1], bmm[1,8] = xi_81, xi_81
bmm[11,1], bmm[1, 11] = xi_11_1, xi_11_1
bmm[2,2] = eta_2 / dt - xi_21 - xi_32 + k_2
bmm[2,3], bmm[3,2] = xi_32, xi_32
bmm[3,3] = eta_3 / dt - xi_32 - xi_43 + k_3
bmm[3,4], bmm[4,3] = xi_43, xi_43
bmm[4,4] = eta_4 / dt - xi_43 + k_4
bmm[5,5] = eta_5 / dt - xi_51 - xi_65 + k_5
bmm[5,6], bmm[6,5] = xi_65, xi_65
bmm[6,6] = eta_6 / dt - xi_65 - xi_76 + k_6
bmm[6,7], bmm[7,6] = xi_76, xi_76
bmm[7,7] = eta_7 / dt - xi_76 + k_7
bmm[8,8] = eta_8 / dt - xi_81 - xi_98 + k_8
bmm[8,9], bmm[9,8] = xi_98, xi_98
bmm[9,9] = eta_9 / dt - xi_98 - xi_10_9 + k_9
bmm[9,10], bmm[10,9] = xi_10_9, xi_10_9
bmm[10,10] = eta_10 / dt - xi_10_9 + k_10
bmm[11,11] = eta_11 / dt - xi_11_1 - xi_12_11 + k_11
bmm[11,12], bmm[12,11] = xi_12_11, xi_12_11
bmm[12,12] = eta_12 / dt - xi_12_11 - xi_13_12 + k_12
bmm[12,13], bmm[13,12] = xi_13_12, xi_13_12
bmm[13,13] = eta_13 / dt - xi_13_12 + k_13

In [31]:
print(bmm)

[[ 10.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  1.  -4.   2.   0.   0.   3.   0.   0.   4.   0.   0.   5.   0.   0.]
 [  0.   2.  19.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   1.  30.   2.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   2.  42.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   3.   0.   0.   0.   7.   1.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   1.  19.   2.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   2.  31.   0.   0.   0.   0.   0.   0.]
 [  0.   4.   0.   0.   0.   0.   0.   0.  39.   1.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   1.   8.   2.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   2.  20.   0.   0.   0.]
 [  0.   5.   0.   0.   0.   0.   0.   0.   0.   0.   0.  27.   1.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.  41.   2.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.

In [97]:
# class for autobuilding the matrix using a class like structure

class Segment:
    def __init__(self, store, env, name="Segment"):
        """builds a segment, that has children and parents
        """
        self.parent = None # should be a segment object or none
        self.childlinks = None # will end up being a list (segment, bloodcoff) tuples
        self.store = store
        self.env = env 
        self.name = name
        
    def add_child(self, blcoeff, child):
        """Add a child to the segment tree
        """
        child.parent = self
        if self.childlinks is None:
            self.childlinks = [(child, blcoeff)]
        else:
            self.childlinks.append((child, blcoeff))
        return self # needed to allow construction of more compicated trees
            
    def build_name_matrix(self):
        """Build a Matrix whoes elemnts are strings describing
        the interactions
        """
        size = self._get_size()
        nmat = np.full((size,size), '')
        cid = -1 # start with -1 since each call sets itself
        _, nmat = self._build_name_matrix(cid, nmat)
        return nmat
    
    def _build_name_matrix(self, cid, nmat):
        """Internal function to build the name matrix as a
        template for building the matrix of the system
        """
        pid = cid
        cid += 1
        if self.childlinks is None:
            # add self term to matrix, and return
            nmat[cid,cid] = "-".join((self.name, "self"))
        else:
            # here we have to preform the recursions
            sid = cid # self ID
            for child, bcf in self.childlinks:
                nmat[sid, cid+1] = "-".join((self.name, child.name))
                nmat[cid+1, sid] = "-".join((self.name, child.name))
                cid, nmat = child._build_name_matrix(cid, nmat)
        return cid, nmat
                
        
    def _get_size(self):
        """Recursively walk the tree in depth first computing
        the size
        """
        if self.childlinks is None:
            return 1
        else:
            size = 1
            for child, bcf in self.childlinks:
                size += child._get_size()
            return size
        

In [98]:
# okay, build a demo body (T, (H, (A, Hn), L))
demob = Segment(1,1,"T")

In [99]:
demob.add_child(0.1, Segment(1,1, "H"))
demob.add_child(0.2, Segment(1,1, "A").add_child(0.3, Segment(2,2, "Hn")))
demob.add_child(0.4, Segment(1,1, "L"))

In [100]:
demob

In [101]:
demob.build_name_matrix()

array([['', 'T', 'T', '', 'T'],
       ['T', 'H', '', '', ''],
       ['T', '', '', 'A', ''],
       ['', '', 'A', 'H', ''],
       ['T', '', '', '', 'L']],
      dtype='<U1')